## GBP/TRY data set

In [49]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [52]:
data_GBP_TR = pd.read_csv("csv files/GBP_TRY.csv")
data_GBP_TR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2892 entries, 0 to 2891
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      2892 non-null   object 
 1   Price     2892 non-null   float64
 2   Open      2892 non-null   float64
 3   High      2892 non-null   float64
 4   Low       2892 non-null   float64
 5   Vol.      1036 non-null   object 
 6   Change %  2892 non-null   object 
dtypes: float64(4), object(3)
memory usage: 158.3+ KB


In [20]:
df = data_GBP_TR.drop(['Open','High','Low','Vol.','Change %'], axis=1)

In [21]:
df = df.rename(columns={'Date': 'Tarih', 'Price': 'Şimdi'})
df

,Tarih,Şimdi
0,12/19/2022,22.6848
1,12/18/2022,22.6635
2,12/16/2022,22.6220
3,12/15/2022,22.6922
4,12/14/2022,23.1319
...,...,...
2887,01/06/2012,2.8974
2888,01/05/2012,2.9166
2889,01/04/2012,2.9342
2890,01/03/2012,2.9305


## LSTM

In [22]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
raw_seq = df['Şimdi']
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[22.6848 22.6635 22.622  22.6922 23.1319] 22.9513
[22.6635 22.622  22.6922 23.1319 22.9513] 22.8621
[22.622  22.6922 23.1319 22.9513 22.8621] 22.8559
[22.6922 23.1319 22.9513 22.8621 22.8559] 22.793
[23.1319 22.9513 22.8621 22.8559 22.793 ] 22.7334
[22.9513 22.8621 22.8559 22.793  22.7334] 22.6039
[22.8621 22.8559 22.793  22.7334 22.6039] 22.6973
[22.8559 22.793  22.7334 22.6039 22.6973] 22.891
[22.793  22.7334 22.6039 22.6973 22.891 ] 22.8066
[22.7334 22.6039 22.6973 22.891  22.8066] 22.4115
[22.6039 22.6973 22.891  22.8066 22.4115] 22.2773
[22.6973 22.891  22.8066 22.4115 22.2773] 22.2543
[22.891  22.8066 22.4115 22.2773 22.2543] 22.475
[22.8066 22.4115 22.2773 22.2543 22.475 ] 22.5229
[22.4115 22.2773 22.2543 22.475  22.5229] 22.4242
[22.2773 22.2543 22.475  22.5229 22.4242] 22.112
[22.2543 22.475  22.5229 22.4242 22.112 ] 22.001
[22.475  22.5229 22.4242 22.112  22.001 ] 22.091
[22.5229 22.4242 22.112  22.001  22.091 ] 22.0158
[22.4242 22.112  22.001  22.091  22.0158] 22.1394
[22.11

In [23]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)
# demonstrate prediction
x_input = array([[df['Şimdi'][0],df['Şimdi'][1],df['Şimdi'][2],df['Şimdi'][3],df['Şimdi'][4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

Epoch 1/200
91/91 [==============================] - 2s 5ms/step - loss: 5.5278
Epoch 2/200
91/91 [==============================] - 0s 5ms/step - loss: 0.0748
Epoch 3/200
91/91 [==============================] - 0s 5ms/step - loss: 0.0716
Epoch 4/200
91/91 [==============================] - 0s 5ms/step - loss: 0.0659
Epoch 5/200
91/91 [==============================] - 0s 5ms/step - loss: 0.0611
Epoch 6/200
91/91 [==============================] - 0s 5ms/step - loss: 0.0683
Epoch 7/200
91/91 [==============================] - 0s 5ms/step - loss: 0.0821
Epoch 8/200
91/91 [==============================] - 0s 5ms/step - loss: 0.0706
Epoch 9/200
91/91 [==============================] - 0s 4ms/step - loss: 0.0635
Epoch 10/200
91/91 [==============================] - 0s 4ms/step - loss: 0.0577
Epoch 11/200
91/91 [==============================] - 0s 5ms/step - loss: 0.0635
Epoch 12/200
91/91 [==============================] - 0s 5ms/step - loss: 0.0662
Epoch 13/200
91/91 [=================

## Daily

In [24]:
print("Prediction:",round(yhat[0][0],3), "₺ Sterlin/TRY")

Prediction: 23.051 ₺ Sterlin/TRY


## Weekly

In [25]:
x_input = array([df['Şimdi'][0], df['Şimdi'][1],df['Şimdi'][2], df['Şimdi'][3], float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_2 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1],df['Şimdi'][2], float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_3 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1],float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_4 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], float(yhat_4),float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_5 = model.predict(x_input, verbose=0)


yhat = float(yhat)
yhat_2 = float(yhat_2)
yhat_3 = float(yhat_3)
yhat_4 = float(yhat_4)
yhat_5 = float(yhat_5)


In [26]:
# onluk olarak yukarı yuvarlandı.
print(f"Pazartesi: {round(yhat,3)} Salı: {round(yhat_2,3)} Carsamba: {round(yhat_3,3)} Persembe: {round(yhat_4,3)} Cuma: {round(yhat_5,3)}  ₺ ₺ Sterlin/TRY")



Pazartesi: 23.051 Salı: 22.964 Carsamba: 22.93 Persembe: 22.932 Cuma: 22.936  ₺ ₺ Sterlin/TRY


## Monthly

In [27]:
import array
exchange = array.array('f', [])

for i in range(len(df)):
    if i % 5 == 0:  # it equals to weekly
        exchange.append(df['Şimdi'][i])


In [41]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[22.6848 22.9513 22.6039 22.2773 22.112 ] 22.058300018310547
[22.9513 22.6039 22.2773 22.112  22.0583] 21.374399185180664
[22.6039 22.2773 22.112  22.0583 21.3744] 21.32069969177246
[22.2773 22.112  22.0583 21.3744 21.3207] 21.316600799560547
[22.112  22.0583 21.3744 21.3207 21.3166] 21.028200149536133
[22.0583 21.3744 21.3207 21.3166 21.0282] 20.377500534057617
[21.3744 21.3207 21.3166 21.0282 20.3775] 21.301700592041016
[21.3207 21.3166 21.0282 20.3775 21.3017] 19.78070068359375
[21.3166 21.0282 20.3775 21.3017 19.7807] 20.825300216674805
[21.0282 20.3775 21.3017 19.7807 20.8253] 20.96299934387207
[20.3775 21.3017 19.7807 20.8253 20.963 ] 20.97410011291504
[21.3017 19.7807 20.8253 20.963  20.9741] 21.171300888061523
[19.7807 20.8253 20.963  20.9741 21.1713] 21.425399780273438
[20.8253 20.963  20.9741 21.1713 21.4254] 21.658000946044922
[20.963  20.9741 21.1713 21.4254 21.658 ] 21.61199951171875
[20.9741 21.1713 21.4254 21.658  21.612 ] 21.815099716186523
[21.1713 21.4254 21.658  21.6

In [44]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(500, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)



Epoch 1/200
18/18 [==============================] - 1s 7ms/step - loss: 23.7566
Epoch 2/200
18/18 [==============================] - 0s 7ms/step - loss: 4.8574
Epoch 3/200
18/18 [==============================] - 0s 7ms/step - loss: 0.9963
Epoch 4/200
18/18 [==============================] - 0s 7ms/step - loss: 0.3102
Epoch 5/200
18/18 [==============================] - 0s 6ms/step - loss: 0.2632
Epoch 6/200
18/18 [==============================] - 0s 6ms/step - loss: 0.2400
Epoch 7/200
18/18 [==============================] - 0s 6ms/step - loss: 0.2236
Epoch 8/200
18/18 [==============================] - 0s 6ms/step - loss: 0.2388
Epoch 9/200
18/18 [==============================] - 0s 6ms/step - loss: 0.2261
Epoch 10/200
18/18 [==============================] - 0s 6ms/step - loss: 0.2095
Epoch 11/200
18/18 [==============================] - 0s 6ms/step - loss: 0.2196
Epoch 12/200
18/18 [==============================] - 0s 6ms/step - loss: 0.2281
Epoch 13/200
18/18 [================

In [46]:
# demonstrate prediction
x_input = array(
    [[exchange[0], exchange[1], exchange[2], exchange[3], exchange[4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)

# onluk olarak yukarı yuvarlandı.
print(f"Next Month -> Prediction: {round(float(yhat[0][0]),3)}  ₺ Sterlin/TRY")


Next Month -> Prediction: 22.112  ₺ Sterlin/TRY
